# Case: Emneord

Et andet spørgsmål er om vi kan bruge genrerummet til at forbedre metadata. En hypotese er emneord ligge i klumper i genrerummet. I så fald kan vi bruge dette til at have et mål for hvor typisk et materiale er for et emne.

Lad os først kigge på hvorledes metadata ser ud:

In [1]:
import bibdata
from collections import Counter
import numpy
bibdata.meta[1234]

{'_id': '870970-basis:51363035',
 'creator': 'Jakob Martin Strid',
 'idx': 1234,
 'language': 'Dansk',
 'subject-term': ['årstider',
  'for 5 år',
  'sk',
  'sjove bøger',
  'Skønlitteratur',
  'for 4 år',
  'for 6 år',
  'for 3 år',
  'venner',
  'vejret',
  'dyrefortællinger'],
 'title': 'Mimbo Jimbo og den lange vinter',
 'type': 'book'}

Som vi kan se er der, for hvert materiale et antal emneord `subject-term`.

Lad os fokusere på de mest populære emneord. Vi kan finde disse ved at lave en liste af alle anvendte emneord `subjects`, og derefter finde de mest hyppige af disse. Listen af alle emneord finder vi ved at gennemløbe alle bibliografiske poster `bib` og for hver af disse tilføje samtlige emneord `subject` til listen af alle emneord `subjects`. Herefter kan den indbyggede `Counter` i Python finde de hyppigst brugte emneord:

In [2]:
subjects = []
for bib in bibdata.meta:
    for subject in bib.get('subject-term', []):
        subjects.append(subject)
Counter(subjects).most_common(10)

[('sk', 7036),
 ('Skønlitteratur', 6445),
 ('for 6 år', 1109),
 ('krimi', 1100),
 ('for 5 år', 1097),
 ('for 7 år', 1063),
 ('let at læse', 1047),
 ('for 4 år', 999),
 ('for 8 år', 932),
 ('Spillefilm', 904)]

Vi er i stand til at finde bøgerne med et givent emneord, ved at gennemløbe alle bøgerne (`book_id`), og kun beholde dem der har det pågældende emneord. Hvis vi eksempelvis vil finde børnefilmene kan vi gøre det således:

In [3]:
films = [book_id for book_id in range(len(bibdata.meta))
         if 'Børnefilm' in bibdata.meta[book_id].get('subject-term', [])]

I koden herover er `len(bibdata.meta)` antallet af materialer, og `range(n)` returnere en liste med `n` tal, i.e. `[0,1,2,...]`. Når vi bruger `.get('subject-term', [])` får vi enten en liste af emneord, eller en tom liste hvis metadata mangler, - ellers ville koden fejle hvis vi manglede metadata.

Nu da vi har listen af børnefilm kan vi finde punkterne i genrerummet for disse:

Vi kan herefter udskrive de først 10 film:

In [4]:
print([bibdata.title_creator(film) for film in films[:10]])

['Bamses billedbog dvd - Katrine Hauch-Fausbøll (movie)', 'Kaj og Andrea - Fausbøll Katrine Hauch- (movie)', 'Ninjago - masters of spinjitzu - Sørensen Thomas inv (movie)', 'Pippi - Hellbom Olle (movie)', 'Filmhits for børn -  (movie)', 'Legends of Chima - Andreasen Tommy inv (movie)', 'Op. Med bonus features disc - Docter Pete (movie)', 'Far til fire - på japansk - Glud Thomas (movie)', 'Ratatouille - Walt Disney firma (movie)', 'Alice i Eventyrland - Walt Disney firma (movie)']


Hver af disse film svare til et punkt(vektor) i genrerummet, som vi kan finde således:

In [5]:
vectors = [bibdata.genres[film] for film in films]

Vi forestiller os at materialerne med samme emneord, ligger i en klump i nærheden af hinanden i genrerummet. I så fald er midten af klumpen gennemsnits-punktet(`mean`), og størrelsen af klumpen den statistiske spredning(`std` = standard deviation), hvilket let findes i Python:

In [6]:
mean = numpy.mean(vectors, axis=0)
std = numpy.std(vectors, axis=0)

Vi kan herefter finde ud af hvor tæt et materiale ligger på klumpen, ved at tage afstanden i genrerummet, vægtet med klumpens størrelse (spredning). I.e. vi tager punktet for materialet(`bibdata.genres[book_id]`), trækker gennemsnitspunktet fra klumpen fra(`mean`), dividere med størrelsen(`std`) og tager længden af dette `numpy.linalg.norm(...)`. Så for alice i eventyrland, cirkeline(616) og turen går til berlin(149) er deres børnefilms-agtighed (jo lavere jo bedre):

In [7]:
print('alice i eventyrland(film)', 
      numpy.linalg.norm((bibdata.genres[810] - mean) / std))
print('cirkeline', 
      numpy.linalg.norm((bibdata.genres[616] - mean) / std))
print('berlin', 
      numpy.linalg.norm((bibdata.genres[149] - mean) / std))

alice i eventyrland(film) 11.79158192
cirkeline 19.9179050997
berlin 38.8215720144


Vi kan nu lave en liste af biblioteksmaterialer, kombineret med hvor meget de minder om børnefilm. Hvis vi derefter sorterer disse, finder vi de mest børnefilms-agtige materialer:

In [8]:
weighted_books = [
    (numpy.linalg.norm((bibdata.genres[book_id]-mean)/std), book_id)
    for book_id in range(len(bibdata.meta))
]
weighted_books.sort()
print(weighted_books[:10])

[(6.4345398281629294, 3801), (6.6494780896369434, 738), (6.6877823144785085, 3216), (6.7020911867051352, 5598), (6.7608479969260493, 2992), (6.7826793078348446, 3866), (6.8973703018034787, 1742), (6.9213674230626134, 3148), (6.9241092655234002, 2211), (6.9706191223710414, 2931)]


&dash; hvilket er følgende materialer:

In [9]:
[bibdata.title_creator(book_id) for (weight, book_id) in weighted_books][0:10]

['Shrek - Elliott Ted aus (movie)',
 'Ratatouille - Walt Disney firma (movie)',
 'Ice age 2 : på tynd is - Carlos Saldanha (movie)',
 'Lilo & Stitch - Walt Disney firma (movie)',
 'Find Nemo - Reynolds David (movie)',
 'Shrek 2 - Vernon Conrad drt (movie)',
 'Toy story 2 - Walt Disney firma (movie)',
 'Junglebogen - Walt Disney firma (movie)',
 'Happy feet - Miller George f. 1945 (movie)',
 'Madagascar - Darnell Eric (movie)']

Nu har vi sorteret biblioteksmaterialerne efter børnefilms-agtighed, - dette kunne være praktisk at kunne, for hvilket som helst emneord. Derfor definere vi en ny funktion, som samler denne funktionalitet:

In [10]:
def books_by_subject(subject):
    books = [book_id 
        for book_id in range(len(bibdata.meta)) 
        if subject in bibdata.meta[book_id].get('subject-term', [])]
    vectors = [bibdata.genres[book] for book in books]
    mean = numpy.mean(vectors, axis=0)
    std = numpy.std(vectors, axis=0)
    weighted_books = [
        (numpy.linalg.norm((bibdata.genres[book_id]-mean)/std), book_id)
        for book_id in range(10000)]
    sorted_books = [
        book_id for (weight, book_id) in sorted(weighted_books)]
    return sorted_books

Hvis vi tager de første materialer fra listen, som ikke har emneordet `Børnefilm` blandt dets emneord(`subject-term`), kan det være at vi finder nogle bibliografiske poster, som kunne beriges med disse metadata:

In [11]:
[bibdata.title_creator(book_id)
  for (weight, book_id) in weighted_books
  if not 'Børnefilm' in bibdata.meta[book_id].get('subject-term', [])][:10]

['Shrek - Elliott Ted aus (movie)',
 'Shrek 2 - Vernon Conrad drt (movie)',
 'Bee movie - det store honningkomplot - Hickner Steve (movie)',
 'Shrek den tredje - Miller Chris instruktør (movie)',
 'Robin Hood - Walt Disney firma (movie)',
 'Koslænget - Oedekerk Steve (movie)',
 'Pinocchio - Walt Disney firma (movie)',
 'Oliver & Company - Walt Disney firma (movie)',
 'Dumbo - Walt Disney firma (movie)',
 'Boog & Elliot 2 - vilde venner mod husdyrene - Carls John (movie)']

Konklusionen fra dette ekperiment er vist, at genrerummet godt kan bruges til at finde kandidater til berigelse af metadata.

Tilsvarende kan vi også finde de Børnefilm, der er mindst børnefilms-agtige:

In [12]:
[bibdata.title_creator(book_id)
  for (weight, book_id) in reversed(weighted_books)
  if 'Børnefilm' in bibdata.meta[book_id].get('subject-term', [])][0:5]

['Star wars - the clone wars - Murphy Scott aus (movie)',
 'Kidnappet - Muasya Vibeke (movie)',
 'Star wars - the clone wars. Sæson 1 - Lucas George (movie)',
 'Inside out - Walt Disney firma (movie)',
 'Eragon - Hugh Johnson (movie)']

Hvilket stadigt er børnefilm, så med dette konkrete eksperiment fandt vi ikke nogle underlige materialer i forhold til emnet.

Det kunne dog være interessant, at lave lignende eksperimenter med andre emneord end "børnefilm", så lad os omskrive koden til en funktion så vi lettere kan ekspementere med det. Her er en funktion, som givet et emneord finder 1) de materialer, som ikke har emneordet, men minder mest om det i genrerummet, og 2) de materialer, som har emneordet, men minder mindst om det:

In [13]:
def subject_outliers(subject):
    print('Without subject "' + subject + '":')
    print('\n'.join(
        ['  ' + bibdata.title_creator(book_id)
         for book_id in books_by_subject(subject) 
         if not subject in bibdata.meta[book_id].get('subject-term', [])
        ][:5]))
    print('With subject "' + subject + '":')
    print('\n'.join(
        ['  ' + bibdata.title_creator(book_id) 
         for book_id in reversed(books_by_subject(subject))
         if subject in bibdata.meta[book_id].get('subject-term', [])
        ][:5]))

subject_outliers('eventyr')

Without subject "eventyr":
  Sjov med feer og alfer - Daniela Dogliani (book)
  Alfer - Moffett Patricia (book)
  Tommelise - Josef Paleček (book)
  Snehvide - Pikka (book)
  Alice i Eventyrland - Walt Disney firma (book)
With subject "eventyr":
  Et sted i nærheden - Cecelia Ahern (book)
  Dragens dom - Bodil Bang Heinemeier (book)
  Aktive fortællinger, rim og remser : dialogisk læsning med børn - Lotte Salling (book)
  Eventyret om Gammelpot og den forheksede tepotte - Thom Roep (book)
  Eventyret om Gammelpot og den sorte kimono - Thom Roep (book)


Hvis vi skal gå videre på opdagelse i materialer, med givne emneord, kan det være praktisk med en liste med de mest almindelige emneord, hvliket findes således:

In [14]:
"; ".join([subject for (subject, count) in Counter(subjects).most_common(200)])

'sk; Skønlitteratur; for 6 år; krimi; for 5 år; for 7 år; let at læse; for 4 år; for 8 år; Spillefilm; Danmark; kærlighed; ÷; for 10 år; for 9 år; for 3 år; 2000-2009; 77.7; for begynderlæsere; for 11 år; USA; for børn; for 12 år; familien; spænding; romaner; kvinder; spændende bøger; England; Sverige; Dansk; 2010-2019; opskrifter; venskab; for 13 år; for 2 år; historie; Biografier af enkelte personer; humor; Børnefilm; 77.74; sjove bøger; for folkeskolen; for 14 år; fantasy; politiromaner; erindringer; 1990-1999; rejseførere; piger; Computerspil; 79.41; 1940-1949; computerspil; drama; børn; Frilæs/0.-2. kl.; Norge; 1980-1989; 1970-1979; for 15 år; København; drenge; Billedbøger sk; Folkeskolen; Del af serie; 1960-1969; bk; unge; Læs let; amerikanske film; 1900-1999; parforhold; eventyr; veninder; Fantasy; ægteskab; dameromaner; 1950-1959; Frankrig; seriemordere; Tyskland; gå på opdagelse; Oplæsning 0.- 2. kl.; den 2. verdenskrig; far-søn forholdet; strikning; 1930-1939; forelskelse; i

## Forslag til metadata for materiale

Nu har vi set hvorledes, vi kan finde materialer, der ligger tæt ved, eller langt fra, et givent emneord. Det kunne også være sjovt at vende det om, og se hvilket emneord, der ligger tæt på et materiale ud fra hvor det ligger i genre rummet.

Første trin er at lave en statistisk model for populære emneord, eksempelvis emneord, der er tildelt til mere end 10 materialer. Lad os allerførst lave en liste over alle emneord, og hvilke materialer der har dem:

In [15]:
subject_dict = {}

for book in bibdata.meta:
    for subject in book.get('subject-term', []):
        subject_dict[subject] = subject_dict.get(subject, []) + [book.get('idx')]

subject_dict = { 
    subject: book_ids 
    for subject, book_ids in subject_dict.items() 
    if len(book_ids) >= 10
}

Vi kan herefter lave finde gennemsnittet og spredningen, for hvert emne.

In [16]:
subjects = []
for subject, book_ids in subject_dict.items():
    vectors = [bibdata.genres[book_id] for book_id in book_ids]
    mean = numpy.mean(vectors, axis=0)
    std = numpy.std(vectors, axis=0)
    std = std / numpy.linalg.norm(std)
    subjects.append({"name": subject, "mean": mean, "std": std})

Givet et `book_id` kan vi herefter sortere emnerne efter hvor godt de passer til en given bog. Herunder tager vi 10 tilfældige bøger, og udskriver hvilke emner der passer bedst til dem i genrerummet, samt viser hvad det er for en bog, og hvilke metadata den har fået tildelt.

Vi finder tilfældige bøger via `random`, og `random.seed(1)` sikre at vi får de samme bøger hver gang vi kører koden. Afstanden mellem bøger og emner bliver fundet på samme vis som tidligere herover. Denne gang er det dog emnerne, og ikke bøgerne, vi sorterer.

In [17]:
import random; random.seed(1) 

for book_id in [random.randint(0,10000) for i in range(10)]:
    sorted_subjects = sorted([(
        numpy.linalg.norm(
            (bibdata.genres[book_id] - subject.get('mean')) / 
                subject.get('std')), 
        subject.get("name")) for subject in subjects])
    
    print(book_id, bibdata.title_creator(book_id))
    print('metadata:', bibdata.meta[book_id].get('subject-term'))
    print('genrerum:', [name for weight, name in sorted_subjects[:10]])
    print()

2201 Den uendelige historie - Michael Ende (audiobook)
metadata: ['sk', 'Eventyrromaner', 'Skønlitteratur']
genrerum: ['Oplæsning 6.- 7. kl.', 'fantastiske fortællinger', 'Fantastisk fortælling', 'Fantastisk fortæl.', 'Ungdomsbog', 'Seriebøger', '6.-7. klasse', 'Mellemskoling', 'parallelle verdener', 'Oplæsning 3.- 5. kl.']

9325 Magia og den forsvundne baby - Rose Impey (book)
metadata: ['for 0-2. klassetrin', 'Lettal 13', 'for folkeskolenfor 0-2. klassetrin', 'for 9 år', 'for 7 år', 'frilæsning', 'Let 13', 'sk', 'for 8 år', 'trylleri', 'Skønlitteratur', 'Folkeskolen', '÷', 'let at læse', 'søskende', 'for folkeskolen', 'Frilæs/0.-2. kl.', 'romaner']
genrerum: ['Læse let blå 2', 'Fril. 0.-2.kl. 95 a', '1. klasse', 'Hekse', 'for 7-8 år', 'hekse', 'Lettal 16', 'Frilæs/0.-2. kl.', 'for 0-2. klassetrin', 'frilæsning']

1033 Min store Peter Pedal findebog - Margret (book)
metadata: ['aber', 'gå på opdagelse', 'for 5 år', 'sk', 'sjove bøger', 'Skønlitteratur', 'for 4 år', 'for 6 år', 'for 3 

Dette eksperiment, giver os både mere indsigt i metadata såvel som genrerum. Når vi ser på det første par bøger, rammer genrerummet ret godt plet:

- "Den Uendelige Historie" - her rammer genrerummets emneord plet, og mere specifikt end de eksisterende metadata. Jeg vil give maskinen ret i at dette er en fantastisk fortælling om parallelle verdener for 6-7 klasse. Vi finder også her at der er forskellige emneord for "fantastiske fortællinger", hvilket kunne give mening at normalisere.
- "Magia" - her er det lidt interessant at metadata er "søskende" og "trylleri", hvor det er "hekse" i genrerummet. Dette minder om at genrerummet afspejler hvad der ellers lånes, hvilket indikere at bogens lånere går efter andre hekse-bøger. En søgning efter bogen, viser en heks på forsiden af bogen.

Det er at bemærke at den også skyder forkert. Eksempelvis: når der er geografi/tidsangivelser/etc. har betydning i metadata, kan dette også dukke op i genrerummet, men ofte skudt forkert, - i.e.: forkerte lande, vikinger omkring år 1200 i stedet for legender om riddere omkring år 400, etc.

Konklusionen er at genrerummet godt kan bruges til inspiration om metadata, men ofte også tager fejl, så man kan ikke automatisk udtrække emneord for materialet.

## Anbefalinger af emneord

Dette leder også til overvejelser at vi kan få anbefalinger til andre emneord ud fra et givent emneord. Afstanden mellem emnerord findes på samme vis som afstandend mellem emneord og bøger, dog med den tilføjelse at der normaliseres med begge emneords spredning.

In [18]:
subject_name = 'fantastiske fortællinger'
subject1 = [subject 
    for subject in subjects 
    if subject.get('name') == subject_name
][0]

sorted_subjects = sorted([(
        numpy.linalg.norm(
            (subject1.get('mean') - subject2.get('mean')) /
             (subject1.get('std') * subject2.get('std'))), 
        subject2.get("name")) for subject2 in subjects])
sorted_subjects[:20]

[(0.0, 'fantastiske fortællinger'),
 (32.589011241465435, 'Fantastisk fortæl.'),
 (37.335710868923528, 'Oplæsning 3.- 5. kl.'),
 (39.926106704591696, 'Oplæsning 6.- 7. kl.'),
 (40.539648464301365, 'Fantastiske fortællinger'),
 (45.438862253162597, 'Fantastisk fortælling'),
 (47.40923136227692, 'Fantastiske fortæll.'),
 (49.158284916890935, 'Eventyr'),
 (51.20541490301153, 'parallelle verdener'),
 (51.598369067958373, 'Mellemskolingen'),
 (51.703630185244045, 'Mellemskoling'),
 (51.875821482898822, '6.-7. klasse'),
 (52.337659955147281, 'Seriebøger'),
 (54.247970145997982, 'Oplæsning 3.-5. kl.'),
 (56.363399583178662, 'Spænding'),
 (56.655407858672753, 'eventyrlige fortællinger'),
 (58.706610226001743, 'Ungdomsbog'),
 (58.729694544002122, 'tro'),
 (58.85151876502978, 'Læs højt'),
 (60.029052682330089, 'Oplæsning 3.- 5. kl')]

Konklusionen af stikprøven herover viser, at genrerummet også kan bruges til at finde relaterede emneord.

## Øvelser

- Find listen af de 100 eller 200 mest populære emneord, i stedet for blot de 10 mest populære.
- Find ud af hvilke andre outliers der er, eksempelvis for emneord som "`sex`"ede bøger, "`tv-serier`", "`Fodbold`", "`Biografier af enkelte personer`", etc. som er outliers, enten fordi de ligner dette emne og ikke har emneorder, - eller har emneordet og ikke ligner de andre indenfor emnet. Kunne det give mening at fjerne/tilføje emneordet for disse?